In [ ]:
import os
import pandas as pd
import librosa

directory = "/content/drive/MyDrive/realtime"
df = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".wav"):
        file_path = os.path.join(directory, filename)
        audio_data, sample_rate = librosa.load(file_path)
        features = librosa.feature.mfcc(y=audio_data, sr=sample_rate)
        flattened_features = features.flatten()
        row = pd.DataFrame([flattened_features], columns=[f"feature_{i}" for i in range(len(flattened_features))])
        df = df.append(row, ignore_index=True)

df.to_csv("/content/real.csv", index=False)


<ipython-input-3-87a9c70795f0>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-3-87a9c70795f0>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


In [ ]:
import pandas as pd
import librosa

df = pd.read_csv('/content/real.csv')

chroma_stft_list = []
spectral_centroid_list = []
spectral_bandwidth_list = []
spectral_rolloff_list = []
rmse_list = []
zero_crossing_rate_list = []
label_list = []

for index, row in df.iterrows():
    existing_features = row.values.tolist()

    chroma_stft = sum(existing_features[:100])
    spectral_centroid = sum(existing_features[100:200])
    spectral_bandwidth = sum(existing_features[200:300])
    spectral_rolloff = sum(existing_features[300:400])
    rmse = sum(existing_features[400:500])
    zero_crossing_rate = sum(existing_features[500:600])
    label = existing_features[-1]

    chroma_stft_list.append(chroma_stft)
    spectral_centroid_list.append(spectral_centroid)
    spectral_bandwidth_list.append(spectral_bandwidth)
    spectral_rolloff_list.append(spectral_rolloff)
    rmse_list.append(rmse)
    zero_crossing_rate_list.append(zero_crossing_rate)
    label_list.append(label)

new_df = pd.DataFrame({
    'chroma_stft': chroma_stft_list,
    'spectral_centroid': spectral_centroid_list,
    'spectral_bandwidth': spectral_bandwidth_list,
    'spectral_rolloff': spectral_rolloff_list,
    'rmse': rmse_list,
    'zero_crossing_rate': zero_crossing_rate_list,
    'label': label_list
})

new_df.to_csv('calculated_features_real.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM

csv_file = '/content/combined2 (1).csv'
train_size = 0.8
df = pd.read_csv(csv_file)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
labels = pd.get_dummies(y).values
X_train, X_test, y_train, y_test = train_test_split(X, labels, train_size=train_size)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(128, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)
predicted_labels = np.argmax(y_pred, axis=1)

for i in range(len(predicted_labels)):
    print('Actual:', np.argmax(y_test[i]), 'Predicted:', predicted_labels[i])


Epoch 1/50
70/70 [==============================] - 6s 22ms/step - loss: 0.5210 - accuracy: 0.9217 - val_loss: 0.1775 - val_accuracy: 0.9640
Epoch 2/50
70/70 [==============================] - 1s 9ms/step - loss: 0.1358 - accuracy: 0.9676 - val_loss: 0.1175 - val_accuracy: 0.9676
Epoch 3/50
70/70 [==============================] - 1s 8ms/step - loss: 0.0989 - accuracy: 0.9762 - val_loss: 0.1010 - val_accuracy: 0.9712
Epoch 4/50
70/70 [==============================] - 1s 9ms/step - loss: 0.0851 - accuracy: 0.9793 - val_loss: 0.0873 - val_accuracy: 0.9730
Epoch 5/50
70/70 [==============================] - 1s 9ms/step - loss: 0.0723 - accuracy: 0.9798 - val_loss: 0.1003 - val_accuracy: 0.9676
Epoch 6/50
70/70 [==============================] - 1s 8ms/step - loss: 0.0694 - accuracy: 0.9829 - val_loss: 0.0740 - val_accuracy: 0.9838
Epoch 7/50
70/70 [==============================] - 1s 9ms/step - loss: 0.0722 - accuracy: 0.9748 - val_loss: 0.0652 - val_accuracy: 0.9838
Epoch 8/50
70/70 [=

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install keras tensorflow


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM

csv_file = '/content/combined2 (1).csv'
train_size = 0.8
df = pd.read_csv(csv_file)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
labels = pd.get_dummies(y).values
X_train, X_test, y_train, y_test = train_test_split(X, labels, train_size=train_size)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(128, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)
predicted_labels = np.argmax(y_pred, axis=1)

for i in range(len(predicted_labels)):
    print('Actual:', np.argmax(y_test[i]), 'Predicted:', predicted_labels[i])

model_weights_file = '/content/model_weights.h5'
model.save_weights(model_weights_file)
print('Model weights saved to:', model_weights_file)


Epoch 1/100
70/70 [==============================] - 8s 46ms/step - loss: 0.5043 - accuracy: 0.8713 - val_loss: 0.2027 - val_accuracy: 0.9442
Epoch 2/100
70/70 [==============================] - 1s 14ms/step - loss: 0.1327 - accuracy: 0.9667 - val_loss: 0.1231 - val_accuracy: 0.9676
Epoch 3/100
70/70 [==============================] - 1s 10ms/step - loss: 0.0858 - accuracy: 0.9793 - val_loss: 0.1044 - val_accuracy: 0.9712
Epoch 4/100
70/70 [==============================] - 1s 9ms/step - loss: 0.0744 - accuracy: 0.9807 - val_loss: 0.1108 - val_accuracy: 0.9748
Epoch 5/100
70/70 [==============================] - 1s 8ms/step - loss: 0.0783 - accuracy: 0.9789 - val_loss: 0.0963 - val_accuracy: 0.9748
Epoch 6/100
70/70 [==============================] - 1s 10ms/step - loss: 0.0730 - accuracy: 0.9825 - val_loss: 0.1428 - val_accuracy: 0.9532
Epoch 7/100
70/70 [==============================] - 1s 10ms/step - loss: 0.0692 - accuracy: 0.9802 - val_loss: 0.1075 - val_accuracy: 0.9730
Epoch 8/

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

new_csv_file = '/content/recorded_data (1).csv'
df_new = pd.read_csv(new_csv_file)
X_new = df_new.iloc[:, :-1].values
X_new = np.reshape(X_new, (X_new.shape[0], 1, X_new.shape[1]))

model = Sequential()
model.add(LSTM(128, input_shape=(1, X_new.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(labels.shape[1], activation='softmax'))

model.load_weights('/content/model_weights.h5')

y_pred_new = model.predict(X_new)
predicted_labels_new = np.argmax(y_pred_new, axis=1)

for i in range(len(predicted_labels_new)):
    print('Predicted:', predicted_labels_new[i])


87/87 [==============================] - 1s 2ms/step
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 3
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 3
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 0
Predicted: 

In [ ]:
import sounddevice as sd
import wavio

# Define parameters for audio capture
sample_rate = 44100  # Sample rate in Hz
duration = 5  # Duration of recording in seconds
output_filename = "recorded_audio.wav"  # Output filename

# Callback function to capture audio data
def audio_callback(indata, frames, time, status):
    recorded_data.append(indata.copy())

# Start audio capture
recorded_data = []
with sd.InputStream(callback=audio_callback, channels=1, samplerate=sample_rate):
    print(f"Recording {duration} seconds of audio...")
    sd.sleep(int(duration * 1000))  # Sleep in milliseconds

# Convert the recorded data to a numpy array
recorded_data = np.concatenate(recorded_data, axis=0)

# Save the recorded audio as a .wav file
wavio.write(output_filename, recorded_data, sample_rate, sampwidth=2)  # 16-bit samples

print(f"Recording saved as {output_filename}")


ModuleNotFoundError: ignored

In [ ]:
```python
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

new_csv_file = '/content/calculated_features_real.csv'
df_new = pd.read_csv(new_csv_file)
X_new = df_new.iloc[:, :-1].values
X_new = np.reshape(X_new, (X_new.shape[0], 1, X_new.shape[1]))

model = Sequential()
model.add(LSTM(128, input_shape=(1, X_new.shape[2]), return_sequences=True))
model.add(LSTM(64))
num_classes = 4
model.add(Dense(num_classes, activation='softmax'))

model.load_weights('/content/model_weights.h5')

y_pred_new = model.predict(X_new)
predicted_labels_new = np.argmax(y_pred_new, axis=1)

for i in range(len(predicted_labels_new)):
    print('Predicted:', predicted_labels_new[i])
```